# Dependencies

In [95]:
# !pip install pyautogui opencv-python easyocr --user

# Libraries

In [96]:
import cv2
import easyocr
import logging
import warnings
import pyautogui
import numpy as np

In [97]:
def take_screenshot():
    screenshot = pyautogui.screenshot()
    screenshot = np.array(screenshot)
    return cv2.cvtColor(screenshot, cv2.COLOR_RGB2BGR)

def get_colors():
    return np.array([
        [81, 215, 170],  # Green light
        [73, 209, 162],  # Green
        [159, 194, 229], # Nude light
        [153, 184, 215], # Nude
        [58, 175, 135]   # line
    ], dtype=np.uint8)

def create_mask_colors(screenshot, colors):
    return [cv2.inRange(screenshot, color, color) for color in colors]

def get_table_contours(masks):
    combined_mask = np.bitwise_or.reduce(masks)
    contours, _ = cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def get_table_dimensions(contours):
    largest_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest_contour)
    return x, y, w, h

In [98]:
screenshot = take_screenshot()
colors = get_colors()
masks = create_mask_colors(screenshot, colors)

table_contours = get_table_contours(masks)
table_dims = get_table_dimensions(table_contours)

square_dims = {
    (450, 360): 45, # Easy
    (540, 420): 30, # Medium
    (600, 500): 25, # Hard
}

square_size = square_dims[(table_dims[2], table_dims[3])]

In [99]:
logging.getLogger().setLevel(logging.ERROR)
reader = easyocr.Reader(['en'])

In [100]:
def get_square(screenshot, table_dims, i, j, square_size):
    dx, dy = j * square_size, i * square_size
    x, y, w, h = (
        table_dims[0] + dx, 
        table_dims[1] + dy, 
        square_size, 
        square_size
    )
    return screenshot[y:y+h, x:x+w]

def scale_image(square, width, inter=cv2.INTER_AREA):
    original_height, original_width = square.shape[:2]
    if original_width == width:
        return square
    
    ratio = width / float(original_width)
    height = int(original_height * ratio)
    
    return cv2.resize(square, (width, height), interpolation=inter)

def str_to_number(str_value):
    try:
        return int(str_value)
    except ValueError:
        return None

def get_number(square):
    results = reader.readtext(square)
    if not results:
        return None

    sorted_results = sorted(results, key=lambda x: x[2], reverse=True)
    return int(sorted_results[0][1])
    
def get_cell_value(square, colors):
    for index, color in enumerate(colors):
        mask = cv2.inRange(square, color, color)
        color_percentage = (np.sum(mask) / (mask.size * 255)) * 100
        
        if color_percentage > 90:
            return 0 if index < 2 else -1
    
    return get_number(square)

In [101]:
_, _, width, height = table_dims
table = np.zeros((height // square_size, width // square_size))

for i in range(len(table)):
    for j in range(len(table[i])):
        square = get_square(screenshot, table_dims, i, j, square_size)
        resized_square = scale_image(square, 5 * square_size)
        table[i, j] = get_cell_value(resized_square, colors[:4])
    print(f"Fila {i + 1} de {n} leida")

Fila 1 de 8 leida
Fila 2 de 8 leida
Fila 3 de 8 leida
Fila 4 de 8 leida
Fila 5 de 8 leida
Fila 6 de 8 leida
Fila 7 de 8 leida
Fila 8 de 8 leida


In [102]:
def show_table(table):
    str_value = ""
    for row in table:
        for col in row:
            value = f" {float(col):.2f}"
            str_value += value + ""
        str_value += "\n"

    print(str_value)

In [103]:
show_table(table)

 -1.00 -1.00 1.00 1.00 1.00 -1.00 -1.00 -1.00 -1.00 -1.00
 -1.00 -1.00 1.00 0.00 1.00 -1.00 -1.00 -1.00 -1.00 -1.00
 -1.00 1.00 2.00 2.00 1.00 1.00 1.00 1.00 -1.00 -1.00
 -1.00 1.00 0.00 1.00 -1.00 1.00 0.00 2.00 2.00 1.00
 1.00 3.00 0.00 2.00 -1.00 1.00 2.00 0.00 0.00 0.00
 0.00 0.00 0.00 1.00 -1.00 -1.00 1.00 0.00 0.00 0.00
 0.00 0.00 0.00 2.00 1.00 1.00 2.00 0.00 0.00 0.00
 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00



In [104]:
def get_cell(table, i, j):
    if 0 <= i < len(table) and 0 <= j < len(table[0]):
        return table[i][j]
    return None

def get_adjacent_cells(table, i, j):
    adjacent_cells = []
    
    for dx, dy in [(-1, -1), (0, -1), (1, -1), (1, 0), (1, 1), (0, 1), (-1, 1), (-1, 0)]:
        ni, nj = i + dy, j + dx
        cell_value = get_cell(table, ni, nj)
        
        if cell_value is not None:
            adjacent_cells.append((ni, nj, cell_value))
    
    return adjacent_cells

def adjacent_cell_with_positive_numbers(table, i, j):
    return [(ni, nj, cell_value) for ni, nj, cell_value in get_adjacent_cells(table, i, j) if cell_value > 0]

def mines_around_cell(table, i, j):
    return sum(1 for ni, nj, cell_value in get_adjacent_cells(table, i, j) if cell_value == -2)

def empty_cells_around(table, i, j):
    return {(ni, nj) for ni, nj, cell_value in get_adjacent_cells(table, i, j) if cell_value == 0}

def heuristic(table, i, j):
    mines_around, empty_cells = 0, set()
    adjacent_cells = adjacent_cell_with_positive_numbers(table, i, j)
    
    if not adjacent_cells or get_cell(table, i, j) != 0:
        return 0
    
    for ni, nj, cell_value in adjacent_cells:
        mines_around += cell_value - mines_around_cell(table, ni, nj)
        empty_cells.update(empty_cells_around(table, ni, nj))
        
    return mines_around / len(empty_cells) if empty_cells else int(mines_around > 0)

In [105]:
def get_heuristics(table):
    heuristics = []
    for i in range(len(table)):
        for j in range(len(table[0])):
            value = heuristic(table, i, j)
            if value != 0:
                heuristics.append((i, j, value))
    return heuristics

def is_a_valid_land_mine(table, land_mine):
    for i in range(len(table)):
        for j in range(len(table[i])):
            cell_value = get_cell(table, i, j)
            if 0 < cell_value < mines_around_cell(table, i, j):
                return False
    return True

def is_possible_to_continue(table):
    for i in range(len(table)):
        for j in range(len(table[i])):
            cell = get_cell(table, i, j)
            if 0 < cell and mines_around_cell(table, i, j) < cell:
                return True
    return False

def best_movement(table):
    movement = None
    min_mines = 9
    
    for i in range(len(table)):
        for j in range(len(table[i])):
            cell = get_cell(table, i, j)
            
            if cell == 0:
                mines = mines_around_cell(table, i, j)
                num_adj_cells = len(adjacent_cell_with_positive_numbers(table, i, j))
                
                if mines < min_mines and num_adj_cells > 0:
                    min_mines = mines
                    movement = (i, j)
    
    return movement

def make_posibilities(table, heuristics_list):
    ordered_heuristics_list = sorted(heuristics_list, key=lambda x: x[2])

    if not is_possible_to_continue(table) or not ordered_heuristics_list:
        return
    
    land_mine = ordered_heuristics_list.pop()
    if is_a_valid_land_mine(table, land_mine):
        i, j, value = land_mine
        table[i][j] = -2
        new_heuristics = get_heuristics(table)
        make_posibilities(table, new_heuristics)
    else:
        make_posibilities(table, ordered_heuristics_list)

In [106]:
heuristics_list = get_heuristics(table)
make_posibilities(table, heuristics_list)
square_i, square_j = best_movement(table)

In [107]:
def save_images(table_dims, i, j, square_size):
    x, y, w, h = table_dims
    sx, sy = j * square_size, i * square_size
    
    table = screenshot[y:y+h, x:x+w]
    cv2.imwrite("./img/table.png", table)
    
    cv2.rectangle(table, (sx, sy), (sx+square_size, sy+square_size), (255, 0, 0), 2)
    cv2.imwrite("./img/best_movement.png", table)

In [108]:
save_images(table_dims, square_i, square_j, square_size)